In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [3]:
# Tranform csv data file into a dataframe
main_df = pd.read_csv("IPPS_data.csv")
main_df = pd.DataFrame(main_df)
main_df.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,$32963.07,$5777.24,$4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,$15131.85,$5787.57,$4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,$37560.37,$5434.95,$4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,$13998.28,$5417.56,$4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,$31633.27,$5658.33,$4851.44


## Review data

In [75]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163065 entries, 0 to 163064
Data columns (total 12 columns):
DRG Definition                          163065 non-null object
Provider Id                             163065 non-null int64
Provider Name                           163065 non-null object
Provider Street Address                 163065 non-null object
Provider City                           163065 non-null object
Provider State                          163065 non-null object
Provider Zip Code                       163065 non-null int64
Hospital Referral Region Description    163065 non-null object
 Total Discharges                       163065 non-null int64
 Average Covered Charges                163065 non-null object
 Average Total Payments                 163065 non-null object
Average Medicare Payments               163065 non-null object
dtypes: int64(3), object(9)
memory usage: 14.9+ MB


In [81]:
main_df.columns

Index(['DRG Definition', 'Provider Id', 'Provider Name',
       'Provider Street Address', 'Provider City', 'Provider State',
       'Provider Zip Code', 'Hospital Referral Region Description',
       ' Total Discharges ', ' Average Covered Charges ',
       ' Average Total Payments ', 'Average Medicare Payments'],
      dtype='object')

In [4]:
# Rename columns
main_df = main_df.rename(columns={"Hospital Referral Region Description":"HRR Description"," Average Covered Charges ":"ACC"," Average Total Payments ":"ATP"
                                  ,"Average Medicare Payments":"AMP"})
main_df.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,HRR Description,Total Discharges,ACC,ATP,AMP
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,$32963.07,$5777.24,$4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,$15131.85,$5787.57,$4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,$37560.37,$5434.95,$4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,$13998.28,$5417.56,$4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,$31633.27,$5658.33,$4851.44


## Check data

Check whether we can use the data in its current format.

### Long column names, amounts presented as strings

+ there's 3 columns with amounts. Amounts are preceeded with a dollar sign.
  remove dollar sign so one can work with floats
+ change data type for columns involved to float
+ rename long columns names 

In [5]:
# Data cleansing
## Strip dollar sign
cols = ["ACC","ATP","AMP"]
main_df[cols] = main_df[cols].replace({'\$':''}, regex = True)

## Convert columns to float
main_df[cols] = main_df[cols].astype(float).fillna(0.0)

### Check for null values

In [112]:
# Data investigation
## Null values:
main_df.isnull().sum().sum()

0

## Explore data

Explore data to find out what we can do with it.

In [86]:
# Unique DRG definitions
main_df["DRG Definition"].nunique()

100

In [92]:
# Unique HRR Descriptions
main_df["HRR Description"].nunique()

306

In [93]:
# Unique providers
main_df["Provider Id"].nunique()

3337

In [122]:
# Unique states
main_df["Provider State"].nunique()

51

In [121]:
# Some groupings
## Total ACC, ATP and AMP by provider
x = main_df.groupby("Provider Id").agg({"ACC":"sum", "ATP":"sum", "AMP":"sum"}) 
print(x.head())
## Total ACC, ATP and AMP by provider state
y = main_df.groupby("Provider State").agg({"ACC":"sum", "ATP":"sum", "AMP":"sum"})
print(y.head())

                    ACC        ATP        AMP
Provider Id                                  
10001        3242726.59  804910.31  706395.72
10005         806103.51  333794.43  283887.94
10006        3177042.74  704962.46  614552.52
10007         265749.81  106938.24   89034.56
10008         145338.77   53083.23   44671.93
                         ACC           ATP           AMP
Provider State                                          
AK              9.320560e+06  3.366222e+06  2.993522e+06
AL              1.138353e+08  2.751052e+07  2.332946e+07
AR              5.410275e+07  1.657579e+07  1.430306e+07
AZ              1.174614e+08  2.895056e+07  2.516212e+07
CA              8.819326e+08  1.649940e+08  1.501626e+08


In [8]:
main_df.to_csv("main data cleaned.csv", encoding = "utf-8-sig", index = False)

# Import cleaned data file

In [ ]:
main_df = pd.read_csv("main data cleaned.csv")